## AUTOENCODERS - RECOMMENDATION SYSTEM

### Importing the Libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

### Importing the Dataset

In [3]:
# We Won't be using this dataset
movies = pd.read_csv("ml-1m/movies.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")
users = pd.read_csv("ml-1m/users.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")
ratings = pd.read_csv("ml-1m/ratings.dat", sep = "::", header = None, engine = "python", encoding = "latin-1")

In [4]:
movies

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [5]:
users

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [6]:
ratings

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


### Preparing the Training set and Test set

In [7]:
training_set = pd.read_csv("ml-100k/u1.base", delimiter = "\t")
training_set = np.array(training_set, dtype = "int")

test_set = pd.read_csv("ml-100k/u1.test", delimiter = "\t")
test_set = np.array(test_set, dtype = "int")

In [8]:
pd.DataFrame(training_set)

,0,1,2,3
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561
...,...,...,...,...
79994,943,1067,2,875501756
79995,943,1074,4,888640250
79996,943,1188,3,888640250
79997,943,1228,3,888640275


In [9]:
pd.DataFrame(test_set)

,0,1,2,3
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883
...,...,...,...,...
19994,458,648,4,886395899
19995,458,1101,4,886397931
19996,459,934,3,879563639
19997,460,10,3,882912371


### Getting the no. of users and movies

In [10]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

print(nb_users, nb_movies)

943 1682


### Converting the data into an array with users in lines and movies in columns

In [11]:
def convert(data):
    # Create a list of lists (943 users to be precise)
    new_data = list()
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        user_rating = np.zeros(nb_movies)
        
        # IMPORTANT PYTHON TRICK TO COMBINE
        user_rating[id_movies - 1] = id_ratings
        new_data.append(list(user_rating))
    return new_data


# Apply the conversion function on training set and test set
training_set = convert(training_set)
test_set = convert(test_set)

In [12]:
pd.DataFrame(training_set)

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
939,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
pd.DataFrame(test_set)

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Convert the data into Torch Tensors

In [14]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [15]:
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [16]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

### Creating the Architecture of the Neural Network

In [17]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        
        # Full Connection between input features and 20 nodes in first hidden layer
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        
        # Use the sigmoid activation function
        self.activation = nn.Sigmoid()
    
    def forward(self, x):
        # x -> Input vector, encoding it twice and decoding it twice
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x


sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [18]:
sae

SAE(
  (fc1): Linear(in_features=1682, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=1682, bias=True)
  (activation): Sigmoid()
)

## Training the SAE

In [19]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    # Loop over users
    for id_user in range(nb_users):
        # Get the input vector of features for that user - 
        
        # Add fake dimension at index 0
        input_vector = Variable(training_set[id_user]).unsqueeze(0)
        target = input_vector.clone()
        
        # Add condition to optimize the memory
        if torch.sum(target.data > 0) > 0:
            output_vector = sae(input_vector)
            
            # DO not compute gradient with respect to target
            target.require_grad = False
            output_vector[target == 0] = 0
            loss = criterion(output_vector, target)
            
            # Represnts avg. of the error but by only considering the movies that were rated.
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            
            # To increase or decrease the weights
            loss.backward()
            
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.
            
            # Decides the intensity
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+ str(train_loss/s))

epoch: 1 loss: tensor(1.7717)
epoch: 2 loss: tensor(1.0966)
epoch: 3 loss: tensor(1.0535)
epoch: 4 loss: tensor(1.0384)
epoch: 5 loss: tensor(1.0308)
epoch: 6 loss: tensor(1.0267)
epoch: 7 loss: tensor(1.0237)
epoch: 8 loss: tensor(1.0220)
epoch: 9 loss: tensor(1.0208)
epoch: 10 loss: tensor(1.0196)
epoch: 11 loss: tensor(1.0190)
epoch: 12 loss: tensor(1.0185)
epoch: 13 loss: tensor(1.0180)
epoch: 14 loss: tensor(1.0175)
epoch: 15 loss: tensor(1.0173)
epoch: 16 loss: tensor(1.0169)
epoch: 17 loss: tensor(1.0167)
epoch: 18 loss: tensor(1.0164)
epoch: 19 loss: tensor(1.0163)
epoch: 20 loss: tensor(1.0161)
epoch: 21 loss: tensor(1.0161)
epoch: 22 loss: tensor(1.0159)
epoch: 23 loss: tensor(1.0156)
epoch: 24 loss: tensor(1.0158)
epoch: 25 loss: tensor(1.0155)
epoch: 26 loss: tensor(1.0158)
epoch: 27 loss: tensor(1.0152)
epoch: 28 loss: tensor(1.0152)
epoch: 29 loss: tensor(1.0126)
epoch: 30 loss: tensor(1.0122)
epoch: 31 loss: tensor(1.0098)
epoch: 32 loss: tensor(1.0097)
epoch: 33 loss: t

## Testing the SAE

In [20]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input_vector = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input_vector)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+ str(test_loss/s))

test loss: tensor(0.9588)
